In [ ]:
pip install trl tensorflow-text tf-models-official wandb==0.12.9

     |████████████████████████████████| 4.9 MB 4.5 MB/s 
     |████████████████████████████████| 2.2 MB 42.5 MB/s 
     |████████████████████████████████| 1.7 MB 35.8 MB/s 
     |████████████████████████████████| 181 kB 49.3 MB/s 
     |████████████████████████████████| 145 kB 32.0 MB/s 
     |████████████████████████████████| 97 kB 6.8 MB/s 
     |████████████████████████████████| 63 kB 683 kB/s 
     |████████████████████████████████| 1.8 MB 20.3 MB/s 
     |████████████████████████████████| 3.3 MB 46.1 MB/s 
     |████████████████████████████████| 880 kB 47.7 MB/s 
     |████████████████████████████████| 462 kB 43.8 MB/s 
     |████████████████████████████████| 99 kB 8.7 MB/s 
     |████████████████████████████████| 636 kB 44.5 MB/s 
     |████████████████████████████████| 237 kB 42.5 MB/s 
     |████████████████████████████████| 352 kB 47.1 MB/s 
     |████████████████████████████████| 47.8 MB 62 kB/s 
     |████████████████████████████████| 90 kB 9.8 MB/s 
     |██████████████████

In [ ]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import wandb
import torch
from itertools import chain
from torch.utils.data import DataLoader
from torch.nn import functional as F
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import copy as deepcopy
import os
import pandas as pd
from nltk.cluster import cosine_distance
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, get_polynomial_decay_schedule_with_warmup

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
path_to_repo = '/content/drive/MyDrive/UB-RL'
os.chdir(path_to_repo+'/EDG_DeepRL')

# co_path = "/content/drive/MyDrive/UB-RL/cls_roberta-large_supervised_shuffle.bin"
er_path = "/content/drive/MyDrive/UB-RL/new_ER_model.pth"
ip_path = "/content/drive/MyDrive/UB-RL/new_IP_model.pth"
ex_path = "/content/drive/MyDrive/UB-RL/new_EX_model.pth"

In [ ]:
from src.model import *
from src.rewards import *

ER path: /content/drive/MyDrive/UB-RL/new_ER_model.pth
IP path: /content/drive/MyDrive/UB-RL/new_IP_model.pth
EX path: /content/drive/MyDrive/UB-RL/new_EX_model.pth
No GPU available, using the CPU instead.


Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Empathy Model Loaded


In [ ]:
tokenizer_dict = load_tokenizer()

Loading the tokenizer...


Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
tokenizer = tokenizer_dict['tok']

In [ ]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8' 

map_name_to_handle = {
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
}

map_model_to_preprocess = {
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

In [ ]:
def build_encoder():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=False, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']

  return tf.keras.Model(text_input, net)

In [ ]:
encoder = build_encoder()

INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'.
INFO:absl:Downloaded https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3, Total size: 1.96MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'.
INFO:absl:Downloaded https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1, Total size: 115.55MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'.


In [ ]:
def calc_reward(seeker_post, generated_response, tokenizer, original_response = None, teacher_forcing = False, print_rewards=False):

    # Given sentence, generated response and original response calculate the reward.

    emp_score = calc_empathy_score(seeker_post, generated_response)
    flow_score = informationFlow([generated_response])
    if(teacher_forcing):
      orig_res_enc = encoder(tf.constant(original_response)).numpy().squeeze()
      gen_res_enc = encoder(tf.constant(generated_response)).numpy().squeeze()
      jacc_score = (1-edit_level_jaccard(original_response, generated_response))/(1+edit_level_jaccard(original_response, generated_response))
      cos_dist = (1+cosine_distance(orig_res_enc, gen_res_enc))/(1-cosine_distance(orig_res_enc, gen_res_enc))
      emp_score = emp_score - calc_empathy_score(seeker_post, original_response)
      if(print_rewards):
        print(f"jacc_dist: {jacc_score}, cos_dist: {cos_dist}, emp: {emp_score}")

      return 0.7*emp_score + 0.125*flow_score + 0.125*cos_dist + 0.05*jacc_score
    # coh_score = calc_coherence_score(original_responses, generated_responses)
    return 0.825*emp_score + 0.175*flow_score

In [ ]:
calc_reward(["Hi How are you"], ["I'm doing good. Just hanging out."], tokenizer=tokenizer, original_response = ["I'm doing good. how are you?"], teacher_forcing=True)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


jacc_dist: 0.4482758620689655, cos_dist: 1.1595059523012858, emp: -0.5


-0.18264796285889096

In [ ]:
ckpt_path = "/content/drive/MyDrive/UB-RL/gpt2/best_ckpt_epoch=4_valid_loss=2.5959.ckpt"
model = load_model(ckpt_path, tokenizer_dict['vocab_size'])

Loading Model...


In [ ]:
# model.eval()

## Defining the Conversation Class

In [ ]:
class Conversation:

    def __init__(self, seq_gen, tokenizer_dict):
        self.seq_gen = seq_gen
        self.history = list()
        self.tokenizer_dict = tokenizer_dict
        self.input_history = list()

    def step(self, sentence):
        self.history.append("<sp1> "+sentence)
        # input = preprocess(self.history)
        response, _ = next_sentence_eval(self.seq_gen, sentence, self.tokenizer_dict, input_history=self.input_history)
        reward = calc_reward([sentence], [response], self.tokenizer_dict['tok'])
        self.history.append("<sp2> "+response)
        return response
    
    def train(self, seq_gen, episodes = 100, lr=0.001, batch_size = 32, seed = 10):
        pass 

    def render(self):
        for i, sentence in enumerate(self.history):
            if(self.history[i].split()[0]=="<sp1>"):
                print(f"You: {' '.join(sentence.split()[1:])}")
            else:
                print(f"Bot: {' '.join(sentence.split()[1:])}")
    
    def reset(self):
        self.history.clear()
        self.input_history.clear()
        return self.history

In [ ]:
new_conv = Conversation(model, tokenizer_dict)

In [ ]:
new_conv.reset()

[]

In [ ]:
new_conv.step('I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people, we felt like the only people in the world.')

"That's amazing! I'm glad you didn't go too far away from your friend and had fun!"

In [ ]:
new_conv.step('This was a best friend. I miss her.')

"Oh that's good to hear. It is a great idea to always keep in touch and get to know each other in a more meaningful way!"

In [ ]:
new_conv.step("We no longer talk.")

'Well that sounds like a nice change.'

In [ ]:
new_conv.render()

You: I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people, we felt like the only people in the world.
Bot: That's amazing! I'm glad you didn't go too far away from your friend and had fun!
You: This was a best friend. I miss her.
Bot: Oh that's good to hear. It is a great idea to always keep in touch and get to know each other in a more meaningful way!
You: We no longer talk.
Bot: Well that sounds like a nice change.


## Training with PPO

In [ ]:
from transformers import GPT2Tokenizer
from trl.gpt2 import GPT2HeadWithValueModel, respond_to_batch
from trl.ppo import PPOTrainer

vocab_size = tokenizer_dict['vocab_size']
gpt2_model = GPT2HeadWithValueModel.from_pretrained('gpt2')
gpt2_model.resize_token_embeddings(vocab_size)
gpt2_model_ref = GPT2HeadWithValueModel.from_pretrained('gpt2')
gpt2_model_ref.resize_token_embeddings(vocab_size)
# gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'lm_head.weight', 'v_head.summary.bias', 'v_head.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias',

Embedding(50260, 768)

In [ ]:
state_dict = torch.load('RL_model.pth')
gpt2_model.load_state_dict(state_dict, strict=False)
gpt2_model_ref.load_state_dict(state_dict, strict=False)

<All keys matched successfully>

In [ ]:
new_df=pd.read_csv('../processed_dataset.csv')

In [ ]:
new_df.drop(["Unnamed: 0"], axis = 1, inplace = True)
new_df.head()

,query,response
0,I remember going to see the fireworks with my ...,"Was this a friend you were in love with, or ju..."
1,This was a best friend. I miss her.,Where has she gone?
2,We no longer talk.,Oh was this something that happened because of...
3,it feels like hitting to blank wall when i se...,Oh ya? I don't really see how
4,dont you feel so.. its a wonder,I do actually hit blank walls a lot of times b...


In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()

## Supplementary

In [ ]:
def add_generated_response(dataset, tokenizer):
    # get model response
  responses = list()

  for query_txt in tqdm(dataset['query'].to_list()):
    response, _ = next_sentence_eval(model, query_txt, tokenizer_dict, [], device = 'cuda')
    responses.append(response)

  dataset['generated_response'] = responses
  return dataset

In [ ]:
new_df = add_generated_response(new_df[:10000], tokenizer)

  0%|          | 0/10000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
new_df.to_csv('../processed_df_10000.csv', index = False)

## Important

In [ ]:
new_df

,query,response
0,I remember going to see the fireworks with my ...,"Was this a friend you were in love with, or ju..."
1,This was a best friend. I miss her.,Where has she gone?
2,We no longer talk.,Oh was this something that happened because of...
3,it feels like hitting to blank wall when i se...,Oh ya? I don't really see how
4,dont you feel so.. its a wonder,I do actually hit blank walls a lot of times b...
...,...,...
29662,"Thank you, I am trying my best.",If you need to you can always take it again. G...
29663,What a scary night that was.,What happened? Hope you are okay?
29664,"My house burned night earlier, had to get my m...","Omg that is so scary, I am so sorry to hear th..."
29665,I woke up this morning to my wife telling me s...,Oh hey that's awesome! That is awesome right?


In [ ]:
ppo_config = {"steps": 400,
    "batch_size": 4,
    "forward_batch_size": 4,
    "ppo_epochs": 4,   
    "init_kl_coef":0.2,
    "target": 6,
    "horizon":10000,
    "gamma":0.95,
    "vf_coef":.1}

In [ ]:
wandb.init(name='run4', project='gpt2-empathetic_dialogue', config= ppo_config)

wandb: Currently logged in as: bpn (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.16 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [ ]:
# Offline learning
# processed dataset

def train(df, ppo_config):
  ppo_trainer = PPOTrainer(gpt2_model, gpt2_model_ref, **ppo_config)
  # fbs = ppo_config['forward_batch_size']

  for epoch in tqdm(range(int(np.ceil(ppo_config["steps"]/ppo_config['batch_size'])))):
    torch.cuda.empty_cache()
    logs = dict()
    game_data = dict()
    timing = dict()

    df_batch = df.sample(ppo_config['batch_size'])
    game_data['query'] = df_batch['query'].to_list()
    df_batch['query'] = df_batch['query'].apply(lambda x: "<sp1> "+x)
    # print(df_batch['query'].to_list())
    df_tokens = tokenizer(df_batch['query'].to_list(), padding=True, return_tensors='pt')['input_ids']
    # print(df_tokens)
    
    # print(df_batch['tokens'])
    query_tensors = torch.from_numpy(np.array(df_tokens))
    # print(f"query: {query_tensors}")

    # response_tensors = []
    # for i in range(int(ppo_config['batch_size'])):
    response = respond_to_batch(gpt2_model, query_tensors, top_p=0.92)
    # response_tensors.append(response)
    # response_tensors = torch.cat(response_tensors)
    # print(f"response_tensor: {response}")
    game_data['response'] = [tokenizer.decode(response[i], skip_special_tokens=True) for i in range(ppo_config['batch_size'])]
    # print(game_data['response'])

    rewards = []
    for i in range(int(ppo_config['batch_size'])):
      res = calc_reward([game_data['query'][i]], [game_data['response'][i]], tokenizer, [df_batch['response'].iloc[i]], True)
      # print(res)
      rewards.append(res)
    rewards = torch.from_numpy(np.array(rewards))
    # print(f"rewards: {rewards}")

    stats = ppo_trainer.step(query_tensors, response, rewards)

    table_rows = [list(r) for r in zip(game_data['query'], game_data['response'], rewards.cpu().tolist())]
    logs.update({'game_log':wandb.Table(
        columns=['query', 'response', 'reward'],
        rows=table_rows)})
    logs.update(timing)
    logs.update(stats)
    logs['env/reward_mean'] = torch.mean(rewards).cpu().numpy()
    logs['env/reward_std'] = torch.std(rewards).cpu().numpy()
    logs['env/reward_dist'] = rewards.cpu().numpy()
    wandb.log(logs)

In [ ]:
train(new_df, ppo_config)

  0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
torch.save(gpt2_model.state_dict(), 'RL_model_1.pth')

## Evaluation

In [ ]:
test_samples = new_df.sample(16)

In [ ]:
new_conv = Conversation(model, tokenizer_dict)

In [ ]:
new_upg_conv = Conversation(gpt2_model, tokenizer_dict)

In [ ]:
test_samples

,query,response
28331,My brother is lucky to have me that's for sure.,why is that?
8443,I think I did very good at my interview. I hop...,Good luck. That's always nerve racking waiting...
9303,I just read a lot but really wanted to do some...,Being cooped up is the worst. Hopefully next w...
6022,"Yes, I can't wait!",Well I hope you enjoy it!
16251,I forgot my words when giving a speech. I comp...,Oh no. Was it a important speech?
28297,my friend helped me move to uni last week,"awww that's so sweet of them, are you guys close?"
14021,It's worse when you don't really understand wh...,yea i know what you mean
9548,I have been high nutrient vegan for 3 months n...,Congratulation.
25921,"I was very angry, my boss embarrassed me for n...",what'd he do?
6701,I want backpacking in the woods last summer. I...,Sometimes being alone can be a good thing though


In [ ]:
old_responses = list()
old_rewards = list()
for ind, test_sample in test_samples.iterrows():
  new_conv.reset()
  response = new_conv.step(test_sample['query'])
  reward = calc_reward([test_sample['query']], [response], tokenizer, [test_sample['response']], True)
  old_responses.append(response)
  old_rewards.append(reward)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


jacc_dist: 0.29545454545454547, cos_dist: 0.8374399820345503, emp: 1.0
jacc_dist: 0.23333333333333334, cos_dist: 0.9004519121608909, emp: 0.5
jacc_dist: 0.25, cos_dist: 0.7575490324410465, emp: 0.5
jacc_dist: 0.325, cos_dist: 0.7869817932982996, emp: 0.0
jacc_dist: 0.28846153846153844, cos_dist: 0.8607542934959163, emp: 0.0
jacc_dist: 0.2916666666666667, cos_dist: 0.8319709166244167, emp: -1.0
jacc_dist: 0.24, cos_dist: 0.745421019558352, emp: -0.5
jacc_dist: 0.391304347826087, cos_dist: 0.6411207016232937, emp: 0.0
jacc_dist: 0.3, cos_dist: 0.8208622065053843, emp: -0.5
jacc_dist: 0.2391304347826087, cos_dist: 0.8328228530808104, emp: 1.0
jacc_dist: 0.25, cos_dist: 0.5974593996020329, emp: 0.5
jacc_dist: 0.234375, cos_dist: 0.8240252836948473, emp: -0.5
jacc_dist: 0.3148148148148148, cos_dist: 0.7499045867226375, emp: 0.5
jacc_dist: 0.2608695652173913, cos_dist: 0.7451741821967506, emp: 1.0
jacc_dist: 0.20454545454545456, cos_dist: 0.844896905429962, emp: 0.0
jacc_dist: 0.233333333333

In [ ]:
new_responses = list()
new_rewards = list()
for ind, test_sample in test_samples.iterrows():
  new_upg_conv.reset()
  response = new_upg_conv.step(test_sample['query'])
  reward = calc_reward([test_sample['query']], [response], tokenizer, [test_sample['response']], True)
  new_responses.append(response)
  new_rewards.append(reward)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


jacc_dist: 0.32608695652173914, cos_dist: 0.7532679637557373, emp: -0.5
jacc_dist: 0.3148148148148148, cos_dist: 0.8272895629715579, emp: 0.5
jacc_dist: 0.2, cos_dist: 0.7551662270004907, emp: 0.5
jacc_dist: 0.2708333333333333, cos_dist: 0.822840351777569, emp: 0.5
jacc_dist: 0.2037037037037037, cos_dist: 0.899277190669053, emp: 1.0
jacc_dist: 0.21153846153846154, cos_dist: 0.9019891136544806, emp: -0.5
jacc_dist: 0.2619047619047619, cos_dist: 0.783742564076667, emp: -0.5
jacc_dist: 0.34, cos_dist: 0.7151973953616523, emp: 0.0
jacc_dist: 0.2857142857142857, cos_dist: 0.9069830206133886, emp: 0.5
jacc_dist: 0.25925925925925924, cos_dist: 0.7895290360839291, emp: 1.0
jacc_dist: 0.20833333333333334, cos_dist: 0.8446068653488527, emp: 0.0
jacc_dist: 0.19117647058823528, cos_dist: 0.8126183301822535, emp: 1.0
jacc_dist: 0.17857142857142858, cos_dist: 0.8706583920753904, emp: 0.5
jacc_dist: 0.2, cos_dist: 0.7441283097469386, emp: 1.5
jacc_dist: 0.15384615384615385, cos_dist: 0.83733252365501

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
test_samples['old_gen_res'] = old_responses
test_samples['new_gen_res'] = new_responses
test_samples['old_rewards'] = old_rewards
test_samples['new_rewards'] = new_rewards

In [ ]:
mean_old_rewards = np.mean(old_rewards)
mean_new_rewards = np.mean(new_rewards)

In [ ]:
mean_old_rewards, mean_new_rewards

(0.17734894868046885, 0.332922595856447)

In [ ]:
test_samples

,query,response,old_gen_res,new_gen_res,old_rewards,new_rewards
28331,My brother is lucky to have me that's for sure.,why is that?,Thats good to hear! What do you plan to do with your brother?,I think it's always good to see a brother even if he isn't seeing anyone yet.,0.819453,-0.239537
8443,I think I did very good at my interview. I hope they call me back!,Good luck. That's always nerve racking waiting for them to call. I'm sure you did well though.,Oh that's cool. Are you proud of yourself?,That is great!,0.474223,0.469152
9303,I just read a lot but really wanted to do something outdoors.,Being cooped up is the worst. Hopefully next weekend is nice and sunny for you and your friends!,Yeah outdoors? What are you reading?,"I do not like the outdoors either. Sometimes it is nice to read, But also to stay active.",0.457194,0.454396
6022,"Yes, I can't wait!",Well I hope you enjoy it!,Why'd you say that?,Where do you think you'll be staying?,0.114623,0.466397
16251,I forgot my words when giving a speech. I completely messed up!,Oh no. Was it a important speech?,I'm sorry. It must've been difficult.,I didn't mean to but I am kind of sorry to say that. Was there a big area of the speech you couldn't finish?,0.122017,0.822595
28297,my friend helped me move to uni last week,"awww that's so sweet of them, are you guys close?",What is it like living in uni?,That's great. What are some things you did to help you get to uni?,-0.581420,-0.226674
14021,It's worse when you don't really understand what you are seeing.,yea i know what you mean,Some people only want to keep things simple and easy. There's nothing the ordinary man can do about it.,I guess I'm not sure how to respond to it.,-0.244822,-0.238937
9548,I have been high nutrient vegan for 3 months now and got on the scale today only to discover that I have lost 34 pounds! I was shocked!,Congratulation.,Oh wow! I'm so happy to hear that.,I'm happy to hear that! You should stay under a lot of water.,0.099705,0.106400
25921,"I was very angry, my boss embarrassed me for nothing"".",what'd he do?,"Don't be mad, Just hurt.",Was it because you didn't like your work?,-0.232392,0.477659
6701,I want backpacking in the woods last summer. I had a great time but it was strange not seeing any other person for over a week.,Sometimes being alone can be a good thing though,I would have woken up and gone wild at that point.,Oh yeah? What kind of woods did you go exploring?,0.816059,0.811654


In [ ]:
new_upg_conv.step("Hi. How are you doing?")

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


'I am ok. I just got to hang in there.'

In [ ]:
new_upg_conv.step("I am sorry you have to go through it. Is there any way i can help?")

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


'I hope you can. Our company is hiring and I recently started working with some of our fellow high tech employees. We have a lot to learn. If you would like to have an interview, I would love to. I also have a training class on how to speak in German. It should be a great help.'

In [ ]:
new_upg_conv.step("Oh. That'd be great. I recently got laid off and it'll great to find a new one.")

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


"That's great. Would you have to change jobs?"

In [ ]:
new_upg_conv.render()

You: Hi. How are you doing?
Bot: I am ok. I just got to hang in there.
You: I am sorry you have to go through it. Is there any way i can help?
Bot: I hope you can. Our company is hiring and I recently started working with some of our fellow high tech employees. We have a lot to learn. If you would like to have an interview, I would love to. I also have a training class on how to speak in German. It should be a great help.
You: Oh. That'd be great. I recently got laid off and it'll great to find a new one.
Bot: That's great. Would you have to change jobs?


In [ ]:
new_upg_conv.reset()

[]

In [ ]:
new_upg_conv.step("Hi. How are you?")

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


"I'm fine. I'm having a hard time making a decision."

In [ ]:
new_upg_conv.step("i am sad too")

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


"I'm too busy reading. What do you usually read?"

In [ ]:
new_upg_conv.render()

You: Hi. How are you?
Bot: I'm great. I'm just in a bad sweat. How are you doing?
You: I'm kind of in a bad place right now. I'm really depressed.
Bot: I don't blame you! Stress can be a big deal. Your boss is often sitting on the couch with you.
